<a href="https://colab.research.google.com/github/eyaler/skcd/blob/main/skcd_stabdiff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Setup

from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

!pip install plantcv
#!wget https://raw.githubusercontent.com/danforthcenter/plantcv/3063cde35c7c1e379f977baa1048ec881ff64978/plantcv/plantcv/morphology/segment_combine.py -O /usr/local/lib/python3.7/dist-packages/plantcv/plantcv/morphology/segment_combine.py
from plantcv import plantcv as pcv


from google.colab import files
import os
from google.colab.patches import cv2_imshow

import tensorflow as tf
import tensorflow_hub as hub

from six.moves.urllib.request import urlopen
from six import BytesIO

import numpy as np
from PIL import Image
from PIL import ImageColor
from PIL import ImageDraw
from PIL import ImageFont
from PIL import ImageOps

import cv2
from skimage.morphology import skeletonize, thin, medial_axis, opening, dilation
from skimage.segmentation import watershed
from collections import Counter
import time
import io
from IPython import display


module_handle = "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1"
detector = hub.load(module_handle).signatures['default']


!pip install diffusers==0.2.4
!pip install transformers scipy ftfy
from huggingface_hub import notebook_login



In [ ]:
#@title Setup Stable Diffusion
#@markdown You also need to accept the model license before downloading or using the weights. In this post we'll use model version `v1-4`, so you'll need to  visit [its card](https://huggingface.co/CompVis/stable-diffusion-v1-4), read the license and tick the checkbox if you agree. 

#@markdown You have to be a registered user in 🤗 Hugging Face Hub, and you'll also need to use an access token for the code to work. For more information on access tokens, please refer to [this section of the documentation](https://huggingface.co/docs/hub/security-tokens).

#@markdown Run the cell to enter your token and log in 

notebook_login()


In [ ]:
#@title Data
#@markdown folder on google drive or leave empty to upload manually

image_folder = 'skcd/shortlisted/fg'  #@param {type: "string"}

#@markdown folder on google drive or leave empty to save on colab 

cache_folder = 'stable_diffusion_cache'  #@param {type: "string"}

if image_folder:
  image_folder = os.path.join('/content/gdrive/MyDrive', image_folder)
else:
  image_folder = '/content/skcd'
  os.makedirs(image_folder, exist_ok=True)
  %cd image_folder
  uploaded = files.upload()

if cache_folder:
  cache_folder = os.path.join('/content/gdrive/MyDrive', cache_folder)



In [ ]:
import torch
from diffusers import StableDiffusionPipeline

# make sure you're logged in with `huggingface-cli login
kwargs = dict(revision="fp16", torch_dtype=torch.float16, use_auth_token=True)
if cache_folder:
  kwargs['cache_dir'] = cache_folder
pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", **kwargs)
StableDiffusionPipeline.safety_checker = lambda x,y: x, 0
pipe = pipe.to("cuda")

In [ ]:
#@title Detect
output_folder = 'skcd/output' #@param {type: "string"}
limit_to_these_images =  None #@param   # 7,11,22,18
prompt_suffix = 'dreamy galactic pastel background painting by don hertzfeldt'  #@param {type: "string"}
text2img_seed = 0 #@param {type: "integer"}
brightness_factor = 2 #@param {type: "number"} 
closing_steps =   1#@param {type: "integer"}
max_boxes_to_show = 10 #@param {type: "integer"}
score_threshold = 0.002 #@param {type: "number"}
iou_threshold = 0.2 #@param {type: "number"}
min_area_frac = 0 #@param {type: "number"}
max_area_frac = 0.55 #@param {type: "number"}
skeleton_method = 'lee' #@param ['zhang', 'lee', 'thin', 'medial']
max_expand_comp_area_ratio =  3.5#@param {type: "number"}
max_expand_total_area_ratio =  3.5#@param {type: "number"}
ignore_classes = 'book', 'envelope', 'poster', 'whiteboard' #@param
show_classes = 'boy', 'chair', 'clothing', 'furniture', 'girl', 'human', 'human body', 'mammal', 'man', 'person', 'plant', 'tableware', 'toy', 'tree', 'tripod', 'woman' #@param
dilation_steps =   3#@param {type: "integer"}
segregated_merge_classes =  'boy', 'chair', 'clothing', 'furniture', 'girl', 'head', 'human', 'human body', 'human face', 'human head', 'mammal', 'man', 'person', 'plant', 'tableware', 'toy', 'tree', 'tripod', 'woman'#@param
min_part_size_pixels =  0#@param {type: "number"}
min_part_size_height_frac =  0.02#@param {type: "number"}
min_parts = 6#@param {type: "integer"}
abs_eps=0 #@param {type: "number"}
rel_eps=0 #@param {type: "number"}
anim_frames =  120 #@param {type: "integer"}
frame_ms =  40#@param {type: "integer"}
left_hand_amp =  0.9,0#@param 
right_hand_amp =  0,1#@param 
left_hand_speed =  0.5,0#@param
right_hand_speed =  0,0.5#@param
left_hand_phase =  -1,0#@param
right_hand_phase =  0,-1#@param
x_speed =  3,1.2#@param
y_speed =  0#@param 
z_speed = 0.00,0#@param   
vantage_x_frac = 0.75  #@param {type: "number"}
vantage_y_frac = 0.5  #@param {type: "number"}
line_thickness =   0#@param {type: "integer"}

if ignore_classes:
  if isinstance(ignore_classes, str):
    ignore_classes = [ignore_classes]
  ignore_classes = [cls.lower() for cls in ignore_classes]

if show_classes:
  if isinstance(show_classes, str):
    show_classes = [show_classes]
  show_classes = [cls.lower() for cls in show_classes]

if segregated_merge_classes:
  if isinstance(segregated_merge_classes, str):
    segregated_merge_classes = [segregated_merge_classes]
  segregated_merge_classes = [cls.lower() for cls in segregated_merge_classes]


output_folder = os.path.join('/content/gdrive/MyDrive', output_folder)
os.makedirs(output_folder, exist_ok=True)

if isinstance(limit_to_these_images, int):
  limit_to_these_images = [limit_to_these_images]


def draw_bounding_box_on_image(image,
                               y1,
                               x1,
                               y2,
                               x2,
                               color,
                               font,
                               thickness=3,
                               display_str_list=[]):
  """Adds a bounding box to an image."""
  text = Image.new("RGBA", image.size, (255, 255, 255, 0))
  draw = ImageDraw.Draw(text)
  im_width, im_height = image.size
  (left, right, top, bottom) = (x1 * im_width, x2 * im_width,
                                y1 * im_height, y2 * im_height)
  draw.line([(left, top), (left, bottom), (right, bottom), (right, top),
             (left, top)],
            width=thickness,
            fill=color)

  # If the total height of the display strings added to the top of the bounding
  # box exceeds the top of the image, stack the strings below the bounding box
  # instead of above.
  display_str_heights = [font.getsize(ds)[1] for ds in display_str_list]
  display_str_widths = [font.getsize(ds)[0] for ds in display_str_list]
  # Each display_str has a top and bottom margin of 0.05x.
  total_display_str_height = (1 + 2 * 0.05) * sum(display_str_heights)
  total_display_str_width = (1 + 2 * 0.05) * sum(display_str_widths)

  text_bottom = min(bottom + total_display_str_height, im_height)
  text_left = max(min(left, im_width - total_display_str_width), 0)
  # Reverse list and print from bottom to top.
  for display_str in display_str_list[::-1]:
    text_width, text_height = font.getsize(display_str)
    margin = np.ceil(0.05 * text_height)
    draw.rectangle([(text_left, text_bottom - text_height - 2 * margin),
                    (text_left + text_width, text_bottom)],
                   fill=color)
    draw.text((text_left + margin, text_bottom - text_height - margin),
              display_str,
              fill="black",
              font=font)
    text_bottom -= text_height - 2 * margin
  return Image.alpha_composite(image, text)

from plantcv.plantcv import params
from plantcv.plantcv import dilate
from plantcv.plantcv import outputs
from plantcv.plantcv import find_objects
from plantcv.plantcv._debug import _debug

def find_branch_pts(skel_img, mask=None, label="default"):
    """Find branch points in a skeletonized image.
    The branching algorithm was inspired by Jean-Patrick Pommier: https://gist.github.com/jeanpat/5712699

    Inputs:
    skel_img    = Skeletonized image
    mask        = (Optional) binary mask for debugging. If provided, debug image will be overlaid on the mask.
    label        = optional label parameter, modifies the variable name of observations recorded

    Returns:
    branch_pts_img = Image with just branch points, rest 0

    :param skel_img: numpy.ndarray
    :param mask: np.ndarray
    :param label: str
    :return branch_pts_img: numpy.ndarray
    """
    # In a kernel: 1 values line up with 255s, -1s line up with 0s, and 0s correspond to don't care
    # T like branch points
    t1 = np.array([[-1, 1, -1],
                   [1, 1, 1],
                   [-1, 0, -1]])
    t2 = np.array([[1, -1, 1],
                   [-1, 1, -1],
                   [1, -1, 0]])
    t3 = np.rot90(t1)
    t4 = np.rot90(t2)
    t5 = np.rot90(t3)
    t6 = np.rot90(t4)
    t7 = np.rot90(t5)
    t8 = np.rot90(t6)

    # Y like branch points
    y1 = np.array([[1, -1, 1],
                   [0, 1, 0],
                   [0, 1, 0]])
    y2 = np.array([[-1, 1, -1],
                   [1, 1, 0],
                   [-1, 0, 1]])
    y3 = np.rot90(y1)
    y4 = np.rot90(y2)
    y5 = np.rot90(y3)
    y6 = np.rot90(y4)
    y7 = np.rot90(y5)
    y8 = np.rot90(y6)
    
   
    kernels = [t1, t2, t3, t4, t5, t6, t7, t8, y1, y2, y3, y4, y5, y6, y7, y8]

    branch_pts_img = np.zeros(skel_img.shape[:2], dtype=int)

    # Store branch points
    for kernel in kernels:
        branch_pts_img = np.logical_or(cv2.morphologyEx(skel_img, op=cv2.MORPH_HITMISS, kernel=kernel,
                                                        borderType=cv2.BORDER_CONSTANT, borderValue=0), branch_pts_img)

    # Switch type to uint8 rather than bool
    branch_pts_img = branch_pts_img.astype(np.uint8) * 255

    # Store debug
    debug = params.debug
    params.debug = None

    # Make debugging image
    if mask is None:
        dilated_skel = dilate(skel_img, params.line_thickness, 1)
        branch_plot = cv2.cvtColor(dilated_skel, cv2.COLOR_GRAY2RGB)
    else:
        # Make debugging image on mask
        mask_copy = mask.copy()
        branch_plot = cv2.cvtColor(mask_copy, cv2.COLOR_GRAY2RGB)
        skel_obj, skel_hier = find_objects(skel_img, skel_img)
        cv2.drawContours(branch_plot, skel_obj, -1, (150, 150, 150), params.line_thickness, lineType=8,
                         hierarchy=skel_hier)

    branch_objects, _ = find_objects(branch_pts_img, branch_pts_img)

    # Initialize list of tip data points
    branch_list = []
    branch_labels = []
    for i, branch in enumerate(branch_objects):
        x, y = branch.ravel()[:2]
        coord = (int(x), int(y))
        branch_list.append(coord)
        branch_labels.append(i)
        cv2.circle(branch_plot, (x, y), params.line_thickness, (255, 0, 255), -1)

    outputs.add_observation(sample=label, variable='branch_pts',
                            trait='list of branch-point coordinates identified from a skeleton',
                            method='plantcv.plantcv.morphology.find_branch_pts', scale='pixels', datatype=list,
                            value=branch_list, label=branch_labels)

    # Reset debug mode
    params.debug = debug

    _debug(visual=branch_plot, filename=os.path.join(params.debug_outdir, f"{params.device}_branch_pts.png"))

    return branch_pts_img


def draw_boxes(image, boxes, classes, scores, merge_lists):
  """Overlay labeled boxes on an image with formatted scores and label names."""
  colors = [c for c in ImageColor.colormap.values() if isinstance(c, str) and 150 < np.mean(ImageColor.getrgb(c)) < 200]

  try:
    font = ImageFont.truetype("/usr/share/fonts/truetype/liberation/LiberationSansNarrow-Regular.ttf",
                              25)
  except IOError:
    print("Font not found, using default font.")
    font = ImageFont.load_default()

  scores = np.array(scores)
  boxes, classes, scores = zip(*sorted([x for x in zip(boxes, classes, scores)], key=lambda x: (show_classes and x[1] not in show_classes, -x[2],  -area(x[0]))))

  bg = image.copy()
  bg_colors = Counter(tuple(c) for c in bg.reshape(-1,3))
  bg_color = bg_colors.most_common()[0][0]
  print(bg_color)
  results = []
  for _ in range(closing_steps):
    image = opening(image)
  image = np.array(Image.fromarray(image).convert("RGBA"))

  im_height, im_width = image.shape[:2]
  filter_classes = []
  filter_scores = []
  filter_merge_lists = []
  masks = []
  skeletons = []
  objects = []
  ind_skel_parts = []
  tight_boxes = []
  final_boxes = []
  new_image = image.copy()
  all_gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
  for (y1, x1, y2, x2), cls, score, mlst in zip(boxes, classes, scores, merge_lists):
    if score < score_threshold or (show_classes and cls not in show_classes):
        continue
    (left, right, top, bottom) = (int(x1 * im_width), int(x2 * im_width),
                            int(y1 * im_height), int(y2 * im_height))
    grab = image[top:bottom, left:right]
    gray = cv2.cvtColor(grab, cv2.COLOR_RGB2GRAY)
    th, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)
    #binary = all_binary[top:bottom, left:right]
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(binary)
    areas = [[stats[label, cv2.CC_STAT_AREA], label] for label in range(1, num_labels)]
    a, max_label = sorted(areas)[-1]
    mask = (labels == max_label).astype(np.uint8) * 255
    x, y, w, h = cv2.boundingRect(mask)
    box = (top+y) / im_height, (left+x) / im_width, (top+y+h) / im_height, (left+x+w) / im_width
    if (max_expand_comp_area_ratio > 1 or not max_expand_comp_area_ratio) and (max_expand_total_area_ratio > 1 or not max_expand_total_area_ratio):
      all_binary = (all_gray < th).astype(np.uint8) * 255     
      num_all_labels, all_labels, all_stats, all_centroids = cv2.connectedComponentsWithStats(all_binary)
      lst = all_labels[top:bottom, left:right] * (mask > 0)
      lst = [l for l in lst.flatten() if l]
      big_label = max(set(lst), key=lst.count)
      big_mask = (all_labels == big_label).astype(np.uint8) * 255
      x, y, w, h = cv2.boundingRect(big_mask)
      final_box = box
      big_box = y / im_height, x / im_width, (y+h) / im_height, (x+w) / im_width
      if (all_stats[big_label, cv2.CC_STAT_AREA] <= a * max_expand_comp_area_ratio or not max_expand_comp_area_ratio) and (area(big_box) <= area(box) * max_expand_total_area_ratio or not max_expand_total_area_ratio):
        final_box = big_box
        mask = big_mask
        top = left = 0
        bottom = im_height
        right = im_width
    if final_box in final_boxes:
      continue
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if skeleton_method == 'zhang':
      sk = skeletonize(mask)
    elif skeleton_method == 'lee':
      sk = skeletonize(mask, method='lee')
    elif skeleton_method == 'thin':
      sk = thin(mask)
    elif skeleton_method == 'medial':
      sk = medial_axis(mask)
    contour_mask = mask.copy()
    cv2.drawContours(mask, contours, -1, color=255, thickness=cv2.FILLED)
    overlay = new_image[top:bottom, left:right].copy()
    overlay[mask > 0] = 255, 0, 0, 32
    new_image[top:bottom, left:right] = np.array(Image.alpha_composite(Image.fromarray(new_image[top:bottom, left:right]), Image.fromarray(overlay)))
    new_image[top:bottom, left:right][sk > 0] = 0, 255, 255, 255
    dilated_mask = mask
    for _ in range(dilation_steps):
      dilated_mask = dilation(dilated_mask)
    pcv.params.line_thickness = 2
    pcv.params.color_scale = 'gist_ncar'
    pcv.params.color_sequence = 'random'
    #cycle_img = pcv.morphology.check_cycles(sk)
    #num_cycles = pcv.outputs.observations['default']['num_cycles']['value']
    bp = find_branch_pts(sk)
    bp = pcv.dilate(bp, 3, 1)
    segments = pcv.image_subtract(sk, bp)
    segment_parts, _ = pcv.find_objects(segments, segments)
    rand_color = pcv.color_palette(num=len(segment_parts), saved=False)
    segmented_img = sk.copy()
    segmented_img = cv2.cvtColor(segmented_img, cv2.COLOR_GRAY2RGB)
    for q, cnt in enumerate(segment_parts):
        cv2.drawContours(segmented_img, segment_parts, q, rand_color[q], thickness=1)        
    
    h, w = mask.shape
    markers = np.zeros((h, w))
    labels = np.arange(len(segment_parts)) + 1
    for m, l in enumerate(labels):
        cv2.drawContours(markers, segment_parts, m, color=int(l), thickness=1)
    filled_mask = watershed(contour_mask == 0, markers=markers, mask=contour_mask != 0, compactness=0)
    non_black = np.any(segmented_img != (0,0,0), axis=-1)
    new_image[top:bottom, left:right, :3][non_black] = segmented_img[non_black]
    skel_parts = []
    small_skel_parts = []
    other_parts = []
    skel_holes = []
    small_skel_holes = []
    other_holes = []
    
    jj = 0
    for j, part in enumerate(segment_parts, start=1):
      this_mask = (filled_mask == j).astype(np.uint8)
      contours, hierarchy = cv2.findContours(this_mask, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE) 
      arc = cv2.arcLength(part, closed=True) / 2
      if arc < min_part_size_pixels or arc < (bottom-top) * min_part_size_height_frac:
        continue
      else:
        jj += 1
      if jj == 4:
        segment_list = range(j + 1, len(segment_parts))
        #if segment_list:
        #  _, segment_parts = pcv.morphology.segment_combine(segment_list, segment_parts, mask)
        break
    
    jj = 0
    for j, part in enumerate(segment_parts, start=1):
      this_mask = (filled_mask == j).astype(np.uint8)
      contours, hierarchy = cv2.findContours(this_mask, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE) 
      if hierarchy is None:
        continue
      arc = cv2.arcLength(part, closed=True) / 2
      if arc < min_part_size_pixels or arc < (bottom-top) * min_part_size_height_frac:
        if line_thickness:
          small_skel_parts.append((part, -1))
        else:
          for c,h in zip(contours, hierarchy[0]):
            if h[3] == -1:
              small_skel_parts.append((c, 0))
            elif h[3] == 0:
              small_skel_holes.append((c, 1))
      elif jj > 4:
        for c,h in zip(contours, hierarchy[0]):
          if h[3] == -1:
            other_parts.append((c, 0))
          elif h[3] == 0:
            other_holes.append((c, 1))
      else:
        if line_thickness:
          approx = cv2.approxPolyDP(part, max(abs_eps, rel_eps * arc), closed=True)
          skel_parts.append((approx, -1))
        else:
          for c,h in zip(contours, hierarchy[0]):
            if h[3] == -1:
              skel_parts.append((c, 0))
            elif h[3] == 0:
              skel_holes.append((c, 1))            
        jj += 1
    if len(skel_parts) + len(other_parts) >= min_parts:
      tight_boxes.append(box)
      final_boxes.append(final_box)
      filter_classes.append(cls)
      filter_scores.append(score)
      filter_merge_lists.append(mlst)
      masks.append(mask)
      skeletons.append(sk)
      bg[top:bottom, left:right][dilated_mask > 0] = bg_color
      parts = []
      for part, flag in small_skel_parts + small_skel_holes + other_parts + other_holes + skel_parts + skel_holes:
        shifted = np.array([[[left + x[0][0], top + x[0][1]]] for x in part])
        parts.append((shifted, flag))
      objects.append(parts)
      ind_skel_parts.append(len(small_skel_parts) + len(small_skel_holes) + len(other_parts) + len(other_holes))

  image_pil = Image.fromarray(np.array(new_image))
  classes, scores, merge_lists = filter_classes, filter_scores, filter_merge_lists
  ordered = sorted(zip(tight_boxes, final_boxes, classes, scores, merge_lists, masks, skeletons, objects, ind_skel_parts), key=lambda x: (x[1][1] + x[1][3], x[1][0] + x[1][2]))
  if ordered:
    tight_boxes, final_boxes, classes, scores, merge_lists, masks, skeletons, objects, ind_skel_parts = zip(*ordered)
  else:
    objects = []

  for i, (tight_box, final_box, cls, score, merge_list) in enumerate(zip(tight_boxes, final_boxes, classes, scores, merge_lists)):
    if score >= score_threshold:       
      display_str = "{}. {} ({}%)".format(i + 1, cls, round(100 * score, 1))
      line_color = ImageColor.getcolor(colors[hash(cls) % len(colors)], 'RGB') + (192,)
      if i < max_boxes_to_show:
        if tight_box != final_box:
          image_pil = draw_bounding_box_on_image(image_pil, *tight_box, line_color, font, thickness=1)
          line_color = ImageColor.getcolor(colors[hash(cls) % len(colors)], 'RGB') + (192,)
        image_pil = draw_bounding_box_on_image(image_pil, *final_box, line_color, font, display_str_list=[display_str])
      y1, x1, y2, x2 = final_box
      results.append(display_str +  f' {Counter(merge_list).most_common()} {tight_box} {final_box} center_x={(x1+x2)/2*100:.0f}% center_y={(y1+y2)/2*100:.0f}% height={(y2-y1)*100:.0f}% area={area(final_box)*100:.1f}%')

  return np.array(image_pil), results, tight_boxes, final_boxes, masks, skeletons, objects, ind_skel_parts, bg


def load_image(path):
  image = tf.io.read_file(path)
  image = tf.image.decode_image(image, channels=3)
  return image


def area(box):
  return (box[2]-box[0]) * (box[3]-box[1])


def bb_intersection_over_union(boxA, boxB):
    # determine the (x, y)-coordinates of the intersection rectangle
    yA = max(boxA[0], boxB[0])
    xA = max(boxA[1], boxB[1])
    yB = min(boxA[2], boxB[2])
    xB = min(boxA[3], boxB[3])

    # compute the area of intersection rectangle
    interArea = max((xB - xA, 0)) * max((yB - yA), 0)
    if interArea == 0:
        return 0
    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = abs((boxA[2] - boxA[0]) * (boxA[3] - boxA[1]))
    boxBArea = abs((boxB[2] - boxB[0]) * (boxB[3] - boxB[1]))

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)

    # return the intersection over union value
    return iou

import copy
def run_detector(detector, path):
  try:
    image = load_image(path)
  except Exception:
    return
  converted_image = tf.image.convert_image_dtype(image, tf.float32)[tf.newaxis, ...]
  start_time = time.time()
  result = detector(converted_image)
  end_time = time.time()

  result = {key:value.numpy() for key,value in result.items()}

  print("Found %d objects." % len(result["detection_scores"]))
  #print("Inference time: ", end_time-start_time)

  boxes, classes, scores = result["detection_boxes"], result["detection_class_entities"], result["detection_scores"]
  bcs = list((b,c.decode().lower(),s) for b,c,s in zip(boxes, classes, scores) if (not ignore_classes or c.decode().lower() not in ignore_classes) and min_area_frac <= area(b) <= max_area_frac and s >= score_threshold)
  if bcs:
    boxes, classes, scores = zip(*bcs)

  #indices = tf.image.non_max_suppression(
  #  boxes,
  #  scores,
  #  max_boxes,
  #  float(iou_threshold),
  #  float(score_threshold),
  #  name=None
  #)
  #boxes, classes, scores = zip(*[x for i, x in enumerate(zip(boxes, classes, scores)) if i in indices])

  bcs = list(zip(boxes, classes, scores, [[] for _ in range(len(boxes))]))
  while True:
    best_iou = 0
    for i in range(len(bcs) - 1):
      for j in range(i + 1, len(bcs)):
        iou = bb_intersection_over_union(bcs[i][0], bcs[j][0])
        box = np.array([min(bcs[i][0][0], bcs[j][0][0]), min(bcs[i][0][1], bcs[j][0][1]), max(bcs[i][0][2], bcs[j][0][2]), max(bcs[i][0][3], bcs[j][0][3])])
        if iou > best_iou and area(box) <= max_area_frac and (not segregated_merge_classes or (bcs[i][1] in segregated_merge_classes) == (bcs[j][1] in segregated_merge_classes)):
          best_iou = iou
          best_i = i
          best_j = j
    if best_iou >= iou_threshold:
      bcs2 = bcs.pop(best_j)
      bcs1 = bcs.pop(best_i)
      box = np.array([min(bcs1[0][0], bcs2[0][0]), min(bcs1[0][1], bcs2[0][1]), max(bcs1[0][2], bcs2[0][2]), max(bcs1[0][3], bcs2[0][3])])
      a1 = area(bcs1[0])
      a2 = area(bcs2[0])
      if a2 > a1 or a2 == a1 and bcs2[2] > bcs1[2]:
        cls, score = bcs2[1:3]
        other = bcs1[1]
      else:
        cls, score = bcs1[1:3]
        other = bcs2[1]
      bcs.append((box, cls, score, bcs1[3] + bcs2[3] + [other]))
    else:
      break
  boxes, classes, scores, merge_lists = zip(*bcs)

  image_with_boxes, results, tight_boxes, final_boxes, masks, skeletons, objects, ind_skel_parts, bg = draw_boxes(image.numpy(), boxes, classes, scores, merge_lists)
  global left_hand_amp, right_hand_amp, left_hand_speed,right_hand_speed, left_hand_phase,right_hand_phase, x_speed, y_speed, z_speed
  if isinstance(left_hand_amp, (int,float)):
    left_hand_amp = [left_hand_amp]*len(objects)
  if isinstance(right_hand_amp, (int,float)):
    right_hand_amp = [right_hand_amp]*len(objects)
  if isinstance(left_hand_speed, (int,float)):
    left_hand_speed = [left_hand_speed]*len(objects)
  if isinstance(right_hand_speed, (int,float)):
    right_hand_speed = [right_hand_speed]*len(objects)
  if isinstance(left_hand_phase, (int,float)):
    left_hand_phase = [left_hand_phase]*len(objects)
  if isinstance(right_hand_phase, (int,float)):
    right_hand_phase = [right_hand_phase]*len(objects)
  if isinstance(x_speed, (int,float)):
    x_speed = [x_speed]*len(objects)
  if isinstance(y_speed, (int,float)):
    y_speed = [y_speed]*len(objects)
  if isinstance(z_speed, (int,float)):
    z_speed = [z_speed]*len(objects)

  if len(left_hand_amp) < len(objects):
    left_hand_amp += type(left_hand_amp)([0] * (len(objects)-len(left_hand_amp)))
  if len(right_hand_amp) < len(objects):
      right_hand_amp += type(right_hand_amp)([0] * (len(objects)-len(right_hand_amp)))
  if len(left_hand_speed) < len(objects):
      left_hand_speed += type(left_hand_speed)([0] * (len(objects)-len(left_hand_speed)))
  if len(right_hand_speed) < len(objects):
      right_hand_speed += type(right_hand_speed)([0] * (len(objects)-len(right_hand_speed)))
  if len(left_hand_phase) < len(objects):
      left_hand_phase += type(left_hand_phase)([0] * (len(objects)-len(left_hand_phase)))
  if len(right_hand_phase) < len(objects):
      right_hand_phase += type(right_hand_phase)([0] * (len(objects)-len(right_hand_phase)))
  if len(x_speed) < len(objects):
      x_speed += type(x_speed)([0] * (len(objects)-len(x_speed)))
  if len(y_speed) < len(objects):
      y_speed += type(y_speed)([0] * (len(objects)-len(y_speed)))
  if len(z_speed) < len(objects):
      z_speed += type(z_speed)([0] * (len(objects)-len(z_speed)))

  cv_image = np.array(image_with_boxes)
  cv_image[..., :3] = cv_image[..., 2::-1]
  cv2_imshow(cv_image)
  cv_bg = bg.copy()
  cv_bg[..., :3] = cv_bg[..., 2::-1]
  cv2_imshow(cv_bg)
  putback = cv_bg.copy()
  for o, parts in enumerate(objects):
    parts, flags = zip(*parts)
    for i, flag in enumerate(flags):
      cv2.drawContours(putback, parts, i, color=(255,255,255) if flag == 1 else 0, thickness=line_thickness if flag == -1 else cv2.FILLED, lineType=cv2.LINE_AA)
  cv2_imshow(putback)
  frames = []
  obj_th = {}
  obj_r = {}
  
  from torch import autocast
  from PIL import ImageEnhance

  prompt = os.path.splitext(os.path.basename(path))[0] + ", " + prompt_suffix
  print(prompt)
  generator = None
  div_factor = 64
  if bg.shape[1] < bg.shape[0]:
    width = 512
    height = div_factor * round(width * bg.shape[1] / bg.shape[0] / div_factor)
  else:
    height = 512
    width = div_factor * round(height * bg.shape[0] / bg.shape[1] / div_factor)
  if text2img_seed:
    generator = torch.Generator("cuda").manual_seed(text2img_seed)
  with autocast("cuda"):
    bg_img = pipe(prompt, generator=generator, height=height, width=width)["sample"][0]
  bg_img = bg_img.resize((bg.shape[1],bg.shape[0]))
  enhancer = ImageEnhance.Brightness(bg_img)
  bg_img = enhancer.enhance(brightness_factor)
  bg_img = np.array(bg_img)
  bg = np.minimum(bg_img, bg)
    
  for o, (parts, ind) in enumerate(zip(objects, ind_skel_parts)):
    parts, flags = zip(*parts)
    parts = list(parts)
    avg_th = {0:0, 1:0}
    max_r = {0:0, 1:0}
    leg_diff = np.mean(parts[ind], axis=0)[0][0] > np.mean(parts[ind + 1], axis=0)[0][0]
    for j in range(2):
      part = copy.deepcopy(parts[ind + j : ind + j + 1])
      if leg_diff and j==0 or not leg_diff and j==1:
        ref = min(part[0], key=lambda x: (x[0][0], x[0][1]))
      else:
        ref = max(part[0], key=lambda x: (x[0][0], -x[0][1]))
      part[0] = part[0] - ref
      for k in range(len(part[0])):
        r = np.sqrt(part[0][k][0][0]*part[0][k][0][0]+part[0][k][0][1]*part[0][k][0][1])
        th = np.arctan2(part[0][k][0][0], part[0][k][0][1])
        avg_th[j] += abs(th)
        max_r[j] = max(max_r[j], r)
      avg_th[j] = avg_th[j] / len(part[0])
      obj_th[o] = (avg_th[0] + avg_th[1]) / len(parts[ind : ind + 2])
      obj_r[o] = (max_r[0] + max_r[1])/len(parts[ind : ind + 2])
      print(obj_r[o], obj_th[o]*180/np.pi)
  midx = cv_image.shape[1] * vantage_x_frac
  midy = cv_image.shape[0] * vantage_y_frac
  for i in range(anim_frames):
    frames.append(bg.copy())
    z_order = sorted([(j,z_speed[j]) for j in range(len(objects))], key=lambda x:-x[1]) 
    for o, _ in z_order:
      parts, flags = zip(*objects[o])
      parts = list(parts)
      ind = ind_skel_parts[o]
      z_index = 1 + z_speed[o] * i
      if z_index <= 0:
        continue
      for j in range(len(parts)):
        part = copy.deepcopy(parts[j:j+1])
        for k in range(len(part[0])):
          part[0][k][0][0] = np.round((part[0][k][0][0] + i*x_speed[o] - midx) / z_index + midx)
          part[0][k][0][1] = np.round((part[0][k][0][1] + i*y_speed[o] - midy) / z_index + midy)
        
        jj = j - ind

        if len(parts) > ind + 4:
          hand_diff = np.mean(parts[ind + 3], axis=0)[0][0] > np.mean(parts[ind + 4], axis=0)[0][0]
          if jj in (3, 4):
            if hand_diff and jj==3 or not hand_diff and jj==4:
              rev = 1
              ref = min(part[0], key=lambda x: (x[0][0], x[0][1]))
              hand_amp = right_hand_amp
              hand_speed = right_hand_speed
              hand_phase = right_hand_phase
            else:
              rev = -1
              ref = max(part[0], key=lambda x: (x[0][0], -x[0][1]))
              hand_amp = left_hand_amp
              hand_speed = left_hand_speed
              hand_phase = left_hand_phase
            part[0] = part[0] - ref
            for k in range(len(part[0])):
              r = np.sqrt(part[0][k][0][0]*part[0][k][0][0]+part[0][k][0][1]*part[0][k][0][1])
              th = np.arctan2(part[0][k][0][0], part[0][k][0][1])
              angle = th + hand_amp[o] * rev * (np.sin(hand_speed[o] * i + hand_phase[o]*np.pi/2) - np.sin(np.pi*(1 - hand_phase[o]/2)))
              part[0][k][0][0] = np.round(r * np.sin(angle))
              part[0][k][0][1] = np.round(r * np.cos(angle))
            part[0] = part[0] + ref

        if len(parts) > ind + 1:
          leg_diff = np.mean(parts[ind], axis=0)[0][0] > np.mean(parts[ind + 1], axis=0)[0][0]
          if jj in (0, 1):
            if leg_diff and jj==0 or not leg_diff and jj==1:
              ref = min(part[0], key=lambda x: (x[0][0], x[0][1]))
            else:
              ref = max(part[0], key=lambda x: (x[0][0], -x[0][1]))
            part[0] = part[0] - ref
            for k in range(len(part[0])):
              r = np.sqrt(part[0][k][0][0]*part[0][k][0][0]+part[0][k][0][1]*part[0][k][0][1])
              th = np.arctan2(part[0][k][0][0], part[0][k][0][1])
              angle = th * np.cos(x_speed[o]/(obj_th[o]*obj_r[o]) * i) if obj_th[o]*obj_r[o] != 0 else th
              part[0][k][0][0] = np.round(r * np.sin(angle))
              part[0][k][0][1] = np.round(r * np.cos(angle))
            part[0] = part[0] + ref
            
        cv2.drawContours(frames[-1], part, 0, color=(255,255,255) if flags[j] == 1 else 0, thickness=line_thickness if flags[j] == -1 else cv2.FILLED, lineType=cv2.LINE_AA)

    frames[-1] = Image.fromarray(frames[-1])
  
  out = os.path.join(output_folder, os.path.splitext(os.path.basename(path))[0]+'.gif')
  frames[0].save(out, format='GIF', append_images=frames[1:], save_all=True, optimize=True, duration=frame_ms, loop=0)
  display.display(display.Image(out))

  for r in results:
    print(r)
  print()

for i, fn in enumerate(sorted(os.listdir(image_folder))):
  if limit_to_these_images and i not in limit_to_these_images:
    continue
  print(i, fn)
  run_detector(detector, os.path.join(image_folder, fn))

print('finished')
